<font size=6  color= 'white' > <b>[LEPL1507] - PROJET P4</b> <br>
<br><br>
Airson Alexis<br>
Beniffou Ibrahim <br>
Henneaux Lucas <br>
Lemaire Antoine <br>
Smith Marielle <br>
Canon Théo<br>
<div style="text-align: right"> </div>

<br><br>
</font>


This is the version maintaining the essential parts of code to run

Easier to add new features 


<font size=6  color= 'white' > <b> Phase 1 : retrieving data </b> <br>

The aim of this first part is to get arrays of images 

In [ ]:
# CELL 1.1 : accessing to data 
# Needs [ CELL 0 : import ]

# defines variables 

#-----------------------------------------------------------------------------

path_training = 'Retrieving_data\BelgiumTSC_Training\Training'
path_testing = 'Retrieving_data\BelgiumTSC_Testing\Testing'

nbr_class = 62 # len(next(walk(path_training))[1])  counting the number of classes 
print('number of classes : ', nbr_class)

#-----------------------------------------------------------------------------

# resol is the format for the images that we want for convenience

resol = (50,50) # Square image only ! 
print("Resolution of images : ", resol)

In [ ]:
from Retrieving_data.retrieving_data import accessing

# These are iterators 
# See fct accessing in codes of Phase 1 
training_set = accessing(path_training, resol)
test_set = accessing(path_testing, resol) 

In [ ]:
from Retrieving_data.retrieving_data import store

# These are arrays
# See fct store in codes of Phase 1 
X_train, y_train = store(training_set)
X_test, y_test = store(test_set)

In [ ]:
from Retrieving_data.retrieving_data import accessing_2 

training_set = accessing_2(path_training)
test_set = accessing_2(path_testing)

In [ ]:
from Retrieving_data.retrieving_data import store_2 

X_train, y_train = store_2(training_set, [], [], resol)
X_test, y_test = store_2(test_set, [], [], resol)

<font size=6  color= 'white' > <b> Phase 2 : visualisation of the dataset </b> <br>

The aim of this part is to visualise the initial datasets 

In [ ]:
from Visualisation.Visu import number

initial_nbr_train = number(nbr_class, y_train.copy())
initial_nbr_test = number(nbr_class, y_test.copy())

In [ ]:
from Visualisation.Visu import graphs

graphs(nbr_class, y_train.copy(), y_test.copy(), 'y_train', 'y_test')

<font size=6  color= 'white' > <b> Phase 3 : augmentation of the dataset </b> <br>

The aim of this part is to change the sizes of the initial datasets


In [ ]:
from augmentation.aug import first 

X_train_first, y_train_first = first(nbr_class, initial_nbr_train.copy(), initial_nbr_test.copy(), X_train.copy(), y_train.copy(), resol)

In [ ]:
from Visualisation.Visu import graphs, number

nbr_train_first = number(nbr_class, y_train_first)
graphs(nbr_class, y_train_first.copy(), y_test.copy(), 'y_train_first', 'y_test')

In [ ]:
from augmentation.aug import second 

X_train_second, y_train_second = second(nbr_class, initial_nbr_train.copy(), X_train.copy(), y_train.copy(), resol, max(initial_nbr_train))

In [ ]:
from Visualisation.Visu import graphs, number

nbr_train_second = number(nbr_class, y_train_second)
graphs(nbr_class, y_train_second.copy(), y_test.copy(), 'y_train_second', 'y_test')

<font size=6  color= 'white' > <b> Phase 4 : randomization of the dataset </b> <br>

The aim of this part is to avoid overfitting by applying transformations on the images 

In [ ]:
from keras import Sequential
from keras.layers import RandomZoom, RandomRotation
from tensorflow import get_logger

# This cell is about data augmentation 
# We'll randomize our set and do our augmentation with this (cells later)
# We choose to work with the second method of augmentation  

#-----------------------------------------------------------------------------

get_logger().setLevel('ERROR')

data_augmentation = Sequential() 

data_augmentation.add(RandomZoom(0.05))
data_augmentation.add(RandomRotation(0.05)) 

#-----------------------------------------------------------------------------

# We apply the augmentation on our datasets 
augmented_image_train = data_augmentation(X_train_second.copy())
augmented_image_test = data_augmentation(X_test.copy())

#-----------------------------------------------------------------------------

print("Shape of the randomized test set : ", augmented_image_test.shape)
print("Shape of the randomized training set : ", augmented_image_train.shape)

In [ ]:
from random import randint 
from matplotlib.pyplot import figure, imshow

# A little test to see the result of the augmentation 

#-----------------------------------------------------------------------------

index = randint(0,len(X_train_second))
figure()
imshow(X_train_second[index])
figure()
imshow(augmented_image_train[index])